# Assistant-02<br/>Assistant Framework

This notebook showcases a foundational framework to work with Assistants.

### Get an OpenAI client and an Assistant

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [1]:
import common
import oaihelper as framework

## Get an OpenAI client
client = framework.get_openai_client()

## Create an assistant
assistant = framework.create_assistant(client,"Math Tutor","You are a personal math tutor. Write and run code to answer math questions.", [{"type": "code_interpreter"}], common.gpt_deployment_name)

## Test messages and recall the threads across users

In [2]:


new_message = framework.generate_response(client,assistant,"What's 2+2", "123", "John")

new_message = framework.generate_response(client,assistant,"What's x*x?", "456", "Sarah")

new_message = framework.generate_response(client,assistant,"What the value of x, for x=m*y+5, if y=2 and m=-10?", "123", "John")

new_message = framework.generate_response(client,assistant,"What is the value of y, for y=x*x?, if x=2?", "456", "Sarah")

Creating new thread for Victor with wa_id 789
Generated message: The 1001st prime number is 7927.
To Victor: The 1001st prime number is 7927.
Retrieving existing thread for John with wa_id 123
Generated message: The result of \(2 + 2\) is \(4\).
To John: The result of \(2 + 2\) is \(4\).
Retrieving existing thread for Sarah with wa_id 456
Generated message: The expression \( x \times x \) is equal to \( x^2 \) (x squared), which represents the square of the variable x.
To Sarah: The expression \( x \times x \) is equal to \( x^2 \) (x squared), which represents the square of the variable x.
Retrieving existing thread for John with wa_id 123
Generated message: Given \( y = 2 \) and \( m = -10 \), the equation \( x = m \cdot y + 5 \) becomes:

\[ x = (-10) \cdot 2 + 5 \]
\[ x = -20 + 5 \]
\[ x = -15 \]

So, the value of \( x \) is \( -15 \).
To John: Given \( y = 2 \) and \( m = -10 \), the equation \( x = m \cdot y + 5 \) becomes:

\[ x = (-10) \cdot 2 + 5 \]
\[ x = -20 + 5 \]
\[ x = -1

In [2]:
assistant1 = framework.create_assistant(client,"Golang","You are an assistant that can help write and explain Go code.", None, common.gpt_deployment_name)

In [5]:
new_message = framework.generate_response(client,assistant1,"Write a Go application to find the 1001st prime number.", "789", "Victor")

Retrieving existing thread for Victor with wa_id 789
Generated message: Here's a Go application to find the 1001st prime number:

```go
package main

import (
	"fmt"
)

// isPrime checks if the number is a prime number.
func isPrime(number int) bool {
	if number <= 1 {
		return false // 0 and 1 are not prime numbers
	}
	if number <= 3 {
		return true // 2 and 3 are prime numbers
	}
	if number%2 == 0 || number%3 == 0 {
		return false // exclude multiples of 2 and 3
	}

	for i := 5; i*i <= number; i += 6 {
		if number%i == 0 || number%(i+2) == 0 {
			return false // exclude multiples of i
		}
	}

	return true
}

// findNthPrime finds the nth prime number using a brute force method.
func findNthPrime(nth int) int {
	count := 0
	number := 1

	for count < nth {
		number++
		if isPrime(number) {
			count++
		}
	}

	return number
}

func main() {
	nth := 1001
	nthPrime := findNthPrime(nth)
	fmt.Printf("The %dth prime number is: %d\n", nth, nthPrime)
}

```

To run this Go program, follow thes